<a href="https://colab.research.google.com/github/hbp5181/Linear-Model-uisng-homolog-survey-data/blob/main/future_learning(sequence_to_numbers).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inverse folding with ESM-IF1

The ESM-IF1 inverse folding model is built for predicting protein sequences from their backbone atom coordinates. We provide examples here 1) to sample sequence designs for a given structure and 2) to score sequences for a given structure.

Trained with 12M protein structures predicted by AlphaFold2, the ESM-IF1 model consists of invariant geometric input processing layers followed by a sequence-to-sequence transformer, and achieves 51% native sequence recovery on structurally held-out backbones. The model is also trained with span masking to tolerate missing backbone coordinates and therefore can predict sequences for partially masked structures.

See [GitHub README](https://github.com/facebookresearch/esm/tree/main/examples/inverse_folding) for the complete user guide, and see our [bioRxiv pre-print](https://doi.org/10.1101/2022.04.10.487779) for more details.

## Environment setup (colab)
This step might take up to 10 minutes the first time.

If using a local jupyter environment, instead of the following, we recommend configuring a conda environment upon first use in command line:
```
conda create -n inverse python=3.9
conda activate inverse
conda install pytorch cudatoolkit=11.3 -c pytorch
conda install pyg -c pyg -c conda-forge
conda install pip
pip install biotite
pip install git+https://github.com/facebookresearch/esm.git
```

Afterwards, `conda activate inverse` to activate this environment before starting `jupyter notebook`.

Below is the setup for colab notebooks:

We recommend using GPU runtimes on colab (Menu bar -> Runtime -> Change runtime type -> Hardware accelerator -> GPU)

In [1]:
# Colab environment setup

# Install the correct version of Pytorch Geometric.
import torch
import os

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric

# Install esm
!pip install -q git+https://github.com/facebookresearch/esm.git

# Install biotite
!pip install -q biotite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.1/932.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 MB 13.9 MB/s eta 0:00:00


### Verify that pytorch-geometric is correctly installed

If the notebook crashes at the import, there is likely an issue with the version of torch_geometric and torch_sparse being incompatible with the torch version.

In [2]:
## Verify that pytorch-geometric is correctly installed
import torch_geometric
import torch_sparse
from torch_geometric.nn import MessagePassing

## Load model
This steps takes a few minutes for the model to download.

**UPDATE**: It is important to set the model in eval mode through `model = model.eval()` to disable random dropout for optimal performance.

In [3]:
import esm
model, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
model = model.eval()

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm_if1_gvp4_t16_142M_UR50.pt" to /root/.cache/torch/hub/checkpoints/esm_if1_gvp4_t16_142M_UR50.pt
/usr/local/lib/python3.10/dist-packages/esm/pretrained.py:215: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


## Extract encoder output as structure representation
The encoder output may also be used as a representation for the structure.

For a set of input coordinates with L amino acids, the encoder output will have shape L x 512.

In [9]:
! esm-extract esm2_t33_650M_UR50D /content/RBD_aa_aligned.fasta\
  coordoutputRBD.fasta --repr_layers 33 --include mean
! esm-extract esm2_t33_650M_UR50D /content/ACE2_aa_modified.fasta \
  coordoutputACE2.fasta --repr_layers 33 --include mean
! esm-extract esm2_t33_650M_UR50D /content/ancestors_unique.fasta.txt\
  coordoutputRBDa.fasta --repr_layers 33 --include mean



Transferred model to GPU
Read /content/RBD_aa_aligned.fasta with 52 sequences
Processing 1 of 3 batches (20 sequences)
Processing 2 of 3 batches (20 sequences)
Processing 3 of 3 batches (12 sequences)
Transferred model to GPU
Read /content/ACE2_aa_modified.fasta with 62 sequences
Processing 1 of 13 batches (5 sequences)
Processing 2 of 13 batches (5 sequences)
Processing 3 of 13 batches (5 sequences)
Processing 4 of 13 batches (5 sequences)
Processing 5 of 13 batches (5 sequences)
Processing 6 of 13 batches (5 sequences)
Processing 7 of 13 batches (5 sequences)
Processing 8 of 13 batches (5 sequences)
Processing 9 of 13 batches (5 sequences)
Processing 10 of 13 batches (5 sequences)
Processing 11 of 13 batches (5 sequences)
Processing 12 of 13 batches (5 sequences)
Processing 13 of 13 batches (2 sequences)
Transferred model to GPU
Read /content/ancestors_unique.fasta.txt with 34 sequences
Processing 1 of 2 batches (20 sequences)
Processing 2 of 2 batches (14 sequences)


In [10]:
# Specify the folders containing the .pt files
folder_paths = ['/content/coordoutputRBD.fasta', '/content/coordoutputACE2.fasta','/content/coordoutputRBDa.fasta']

# Flatten the list of filenames
pt_files = [os.path.join(folder, f) for folder in folder_paths for f in os.listdir(folder) if f.endswith('.pt')]

# Iterate over each .pt file
for file_path in pt_files:
    # Load the model using torch.load
    model_dict = torch.load(file_path, map_location=torch.device('cpu'))
    for key, value in model_dict.items():
        print(value)


WIV1_KF367457
{33: tensor([ 0.0231, -0.0028, -0.0825,  ..., -0.0678, -0.0402, -0.0350])}
ZC45_MG772933
{33: tensor([ 0.0231, -0.0156, -0.0568,  ...,  0.0006, -0.0960, -0.0460])}
273-2005_DQ648856
{33: tensor([ 0.0495, -0.0209, -0.0423,  ...,  0.0185, -0.1139, -0.0551])}
Rp3_DQ071615
{33: tensor([ 0.0500, -0.0083, -0.0523,  ..., -0.0240, -0.0853, -0.0465])}
HeB2013_KJ473812
{33: tensor([ 0.0485, -0.0260, -0.0434,  ...,  0.0192, -0.1052, -0.0575])}
GX2013_KJ473815
{33: tensor([ 0.0285, -0.0278, -0.0558,  ..., -0.0304, -0.0799, -0.0668])}
BtKY72_KY352407
{33: tensor([ 0.0296, -0.0303, -0.0700,  ..., -0.0195, -0.0567, -0.0671])}
Rf1_DQ412042
{33: tensor([ 0.0473, -0.0279, -0.0498,  ...,  0.0203, -0.1102, -0.0508])}
LYRa11_KF569996
{33: tensor([ 0.0084, -0.0171, -0.0563,  ..., -0.0425, -0.0463, -0.0373])}
Rf4092_KY417145
{33: tensor([ 0.0147, -0.0205, -0.0313,  ...,  0.0061, -0.1000, -0.0434])}
ZXC21_MG772934
{33: tensor([ 0.0150, -0.0062, -0.0693,  ..., -0.0126, -0.0851, -0.0503])}
Pangoli

In [15]:
# Specify the folders containing the .pt files
folder_paths = ['/content/coordoutputRBD.fasta','/content/coordoutputACE2.fasta',
'/content/coordoutputRBDa.fasta']

formatted_dict = {}

# Iterate over each folder
for folder_path in folder_paths:
    # List all files in the folder with .pt extension
    pt_files = [f for f in os.listdir(folder_path) if f.endswith('.pt')]

    # Iterate over each .pt file in the current folder
    for file_name in pt_files:
        # Construct the full path to the file
        file_path = os.path.join(folder_path, file_name)

        # Load the model using torch.load
        model_dict = torch.load(file_path, map_location=torch.device('cpu'))

        # Extract label and tensor values
        label = model_dict['label']
        tensor_values = model_dict['mean_representations'][33].numpy()

        # Include the first three and last three numbers in the tensor
        first_three = ','.join(map(str, tensor_values[:3]))
        last_three = ','.join(map(str, tensor_values[-3:]))
        formatted_dict[label] = f'{first_three} {last_three}'
print(formatted_dict)

{'WIV1_KF367457': '0.023142694,-0.0027605628,-0.08252598 -0.06780611,-0.040215082,-0.035003077', 'ZC45_MG772933': '0.023055341,-0.015602402,-0.056835543 0.0006173855,-0.09598661,-0.046016414', '273-2005_DQ648856': '0.04952701,-0.020896938,-0.042315926 0.018459592,-0.11392996,-0.055139825', 'Rp3_DQ071615': '0.049971074,-0.008294755,-0.052281905 -0.024001537,-0.08532302,-0.046479423', 'HeB2013_KJ473812': '0.04850938,-0.026046049,-0.04342316 0.019246142,-0.10523409,-0.057524525', 'GX2013_KJ473815': '0.02846526,-0.027832128,-0.05575813 -0.030358147,-0.079894,-0.066789865', 'BtKY72_KY352407': '0.029604875,-0.030271351,-0.069989905 -0.019523937,-0.05670124,-0.06707277', 'Rf1_DQ412042': '0.047305156,-0.027902624,-0.049828533 0.020310806,-0.11023044,-0.050829515', 'LYRa11_KF569996': '0.008418533,-0.017130952,-0.056323368 -0.04253885,-0.046308447,-0.037347417', 'Rf4092_KY417145': '0.014718485,-0.020495415,-0.03129319 0.006141618,-0.10003012,-0.043400798', 'ZXC21_MG772934': '0.0150032705,-0.0062

In [16]:
formatted_dict[label] = f'{first_three} {last_three}'
for key, value in formatted_dict.items():
    print(f'{key}: {value}')

WIV1_KF367457: 0.023142694,-0.0027605628,-0.08252598 -0.06780611,-0.040215082,-0.035003077
ZC45_MG772933: 0.023055341,-0.015602402,-0.056835543 0.0006173855,-0.09598661,-0.046016414
273-2005_DQ648856: 0.04952701,-0.020896938,-0.042315926 0.018459592,-0.11392996,-0.055139825
Rp3_DQ071615: 0.049971074,-0.008294755,-0.052281905 -0.024001537,-0.08532302,-0.046479423
HeB2013_KJ473812: 0.04850938,-0.026046049,-0.04342316 0.019246142,-0.10523409,-0.057524525
GX2013_KJ473815: 0.02846526,-0.027832128,-0.05575813 -0.030358147,-0.079894,-0.066789865
BtKY72_KY352407: 0.029604875,-0.030271351,-0.069989905 -0.019523937,-0.05670124,-0.06707277
Rf1_DQ412042: 0.047305156,-0.027902624,-0.049828533 0.020310806,-0.11023044,-0.050829515
LYRa11_KF569996: 0.008418533,-0.017130952,-0.056323368 -0.04253885,-0.046308447,-0.037347417
Rf4092_KY417145: 0.014718485,-0.020495415,-0.03129319 0.006141618,-0.10003012,-0.043400798
ZXC21_MG772934: 0.0150032705,-0.0062314565,-0.06928141 -0.012619139,-0.08510872,-0.0502924